# Lecture Notes on Factor Analysis and I-Vectors
## Abstract 
该文档 提供了FA 因子分析模型的详细的公式, 因子分析模型 认为观测变量实际由低维空间的隐变量**真实数据(真实分布)** 以及噪声生成, 低维空间由V矩阵描述.
因子分析模型, 认为 观测变量 服从 混合高斯模型GMM , 隐变量的先验概率是高斯分布. 
通过因子分析模型, 将一句utt 抽取为固定长度的隐变量 ivector

## Factor Analysis
X = {$x_1$, $x_2$ .. $x_n$} 是 R 维向量训练集合.
因子分析模型认为 某个$x_i$, 是根据如下模型 由 $z_i$ 生成
$$ X_i = m + Vz_i + \epsilon_i\tag{1} $$
m 是 X的均值, 
V 是一个低秩矩阵R X D, 
$z_i$ 是D维 隐变量因子, 服从$N(z|0, I)$,
$\epsilon_i$ 是 残余噪声, 服从$N(z|0, \Sigma)$,

x 的边缘分布为
$$ \begin{eqnarray}
    p(X)& = & \int {p(x|z)p(z)} {\rm d}x \\
    & = & \int {N(x|m + Vz, \Sigma)N(z|0, I)} {\rm d}x\tag{2}  \\
    & = & N(x|m, VV^T + \Sigma)
\end{eqnarray} $$
由此, p(X) 也是一个 高斯分布

## EM 算法
### M-step 最大化 联合概率似然(先计算E步 或M步没关系)
令 参数集合为 $ w = {m , V, \Sigma} $ 
$$ \begin{eqnarray}
    Q(w) & = & E_z ln p(X, Z| w) \\
    & = & E_z\big\{\sum_i {ln p(x_i | z_i, w) p(z_i)}\big\} \\
    & = & E_z\Big\{\sum_i {ln [N(x_i | m + V z_i, \Sigma) N(z_i|0, I)]} \Big\}  
\end{eqnarray} $$



